# ModelV2Overlapping
## Object detection on overlapping cards

In [2]:
!pip -q install ultralytics wandb


In [3]:
from ultralytics import YOLO
from jassair.utils.paths import get_dataset_path, Datasets
import wandb
import time
import torch
import random
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

%matplotlib inline

### Global variable definition

In [4]:
# DATA_DIR = get_dataset_path(Datasets.SWISS)
BATCH_SIZE = 16
NUM_CLASSES = 36

In [5]:
torch.cuda.is_available()

True

Enable W&B logging for Ultralytics in CLI

`yolo settings wandb=True`

Disable Weights & Biases logging

`yolo settings wandb=False`

In [6]:
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aarfu (aarfu-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Model
We can use a singel YOLO model for both, object detection and classification tasks.

In [7]:
MODEL = YOLO("yolo11n.pt") # YOLOv11 large

## Train
imgsz: yolo resizes the input images to a square image. Since jasscards are not square, yolo resizes the long side to 640 and then scales the shorter side proportionally.

In [8]:
# Hyperparameters
# epochs=50
# imgsz=640 # 640 (default) – Balanced performance; 512 – Faster, lower memory usage, 1280+ – Best accuracy for small details but slower.
# iterations=100 # Number of tuning cycles (higher = better tuning). The model tries different values for lr, momentum, weight decay, etc.

HYPERPARAMETERS = {
    "lr0": 1e-3,        # renamed from "lr" to "lr0"
    "weight_decay": 1e-5,  # renamed from "wd"
    "imgsz": 640,
    "epochs": 80,
}

In [57]:
RUN_NAME = f"yolo11l_1to9_36c_ovlp_{time.strftime('%y%m%d-%H%M%S')}"

In [58]:
wandb.init(
    entity="jassair",
    project="ModelV2",  # Specify project
    name=RUN_NAME, # Run name
    config=HYPERPARAMETERS,
)

In [11]:
# Model with all tunable hyperparameters
MODEL.train(
    data="/exchange/dspro2/jassair/s_1to9_36c_ovlp/data.yaml",
    device=0,
    batch=BATCH_SIZE,
    optimizer="Adam",
    momentum=0.937,
    degrees=10,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.001,
    flipud=0.1,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    project="jassair",
    name=RUN_NAME,
    patience=10,
    **HYPERPARAMETERS
)


New https://pypi.org/project/ultralytics/8.3.109 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.108 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/exchange/dspro2/jassair/s_1to9_36c_ovlp/data.yaml, epochs=80, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=jassair, name=yolo11l_s_1to9_36c_ovlp_250416-174411, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

train: Scanning /exchange/dspro2/jassair/s_1to9_36c_ovlp/train/labels... 3600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3600/3600 [00:17<00:00, 207.20it/s]


train: New cache created: /exchange/dspro2/jassair/s_1to9_36c_ovlp/train/labels.cache


val: Scanning /exchange/dspro2/jassair/s_1to9_36c_ovlp/valid/labels... 360 images, 0 backgrounds, 0 corrupt: 100%|██████████| 360/360 [00:01<00:00, 203.43it/s]

val: New cache created: /exchange/dspro2/jassair/s_1to9_36c_ovlp/valid/labels.cache


...

                   all        360       2003      0.978      0.972      0.993      0.935



80 epochs completed in 1.245 hours.
Optimizer stripped from jassair/yolo11l_s_1to9_36c_ovlp_250416-174411/weights/last.pt, 5.5MB
Optimizer stripped from jassair/yolo11l_s_1to9_36c_ovlp_250416-174411/weights/best.pt, 5.5MB

Validating jassair/yolo11l_s_1to9_36c_ovlp_250416-174411/weights/best.pt...
Ultralytics 8.3.108 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
YOLO11n summary (fused): 100 layers, 2,589,172 parameters, 0 gradients, 6.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:02<00:00,  4.17it/s]


                   all        360       2003      0.978      0.972      0.993      0.935
             eichel 10         55         55      0.991      0.945      0.994      0.927
              eichel 6         55         55      0.992      0.982      0.986      0.947
              eichel 7         56         56      0.965      0.975      0.993      0.963
              eichel 8         56         56      0.964      0.961      0.984      0.924
              eichel 9         55         55      0.976      0.964      0.992      0.944
            eichel ass         56         56          1      0.965      0.995      0.959
          eichel konig         55         55      0.999          1      0.995      0.945
           eichel ober         57         57      0.978          1      0.995      0.936
          eichel under         55         55      0.982      0.965      0.992      0.912
               rose 10         56         56      0.961      0.964       0.99      0.942
                rose 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f214cbe9eb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.0380

In [9]:
wandb.finish()

In [24]:
run = wandb.init(
    entity="jassair",
    project="ModelV2",
    name="upload-final-yolov11-model"
)

model_path = "jassair/yolo11l_s_1to9_36c_ovlp_250416-174411/weights/best.pt"

artifact = wandb.Artifact(
    name="yolov11-model",
    type="model",
    description="YOLOv11 model trained on synthetic, 1to9, 36c, overlapping cards",
    metadata={"framework": "Ultralytics YOLO", "version": "v11"}
)

artifact.add_file(model_path)
run.log_artifact(artifact)

run.finish()


## Finetune on real life images

In [59]:
model = YOLO("jassair/yolo11l_s_1to9_36c_ovlp_250416-174411/weights/best.pt")

In [60]:
# Freeze backbone layers in the Sequential model
for i, (name, module) in enumerate(model.model.model.named_children()):
    if i <= 8:
        for param in module.parameters():
            param.requires_grad = False

In [61]:
epochs = 50
batch_size = 16

HYPERPARAMETERS = {
    "lr0": 1e-3,        # renamed from "lr" to "lr0"
    "weight_decay": 1e-5,  # renamed from "wd"
    "imgsz": 640,
    "epochs": epochs,
}

In [62]:
wandb.init(
    entity="jassair",
    project="ModelV2",  # Specify project
    name=RUN_NAME + 'Finetuned', # Run name
    config=HYPERPARAMETERS,
)

In [63]:
# Model with all tunable hyperparameters
model.train(
    data='/exchange/dspro2/jassair/r_3to5_36c_ovlp/data.yaml',
    device=0,
    batch=BATCH_SIZE,
    optimizer="Adam",
    momentum=0.937,
    degrees=10,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.001,
    flipud=0.1,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    project="jassair",
    name=RUN_NAME + 'Finetuned',
    patience=10,
    **HYPERPARAMETERS
)

New https://pypi.org/project/ultralytics/8.3.115 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.114 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: task=detect, mode=train, model=jassair/yolo11l_s_1to9_36c_ovlp_250416-174411/weights/best.pt, data=/exchange/dspro2/jassair/r_3to5_36c_ovlp/data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=jassair, name=yolo11l_1to9_36c_ovlp_250423-200052Finetuned, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=

train: Scanning /exchange/dspro2/jassair/r_3to5_36c_ovlp/train/labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 716.6±193.9 MB/s, size: 45.2 KB)


val: Scanning /exchange/dspro2/jassair/r_3to5_36c_ovlp/valid/labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]


Plotting labels to jassair/yolo11l_1to9_36c_ovlp_250423-200052Finetuned/labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=1e-05), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to jassair/yolo11l_1to9_36c_ovlp_250423-200052Finetuned
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all          5         22     0.0884      0.452      0.212      0.057
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 2, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



12 epochs completed in 0.003 hours.
Optimizer stripped from jassair/yolo11l_1to9_36c_ovlp_250423-200052Finetuned/weights/last.pt, 5.5MB
Optimizer stripped from jassair/yolo11l_1to9_36c_ovlp_250423-200052Finetuned/weights/best.pt, 5.5MB

Validating jassair/yolo11l_1to9_36c_ovlp_250423-200052Finetuned/weights/best.pt...
Ultralytics 8.3.114 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
YOLO11n summary (fused): 100 layers, 2,589,172 parameters, 0 gradients, 6.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 23.73it/s]


                   all          5         22     0.0894       0.69      0.329      0.251
             Eichel 10          2          2      0.333        0.5       0.58      0.522
              Eichel 7          1          1      0.333          1      0.995      0.796
              Eichel 8          1          1        0.2          1      0.249     0.0746
              Eichel 9          1          1          0          0          0          0
            Eichel Ass          1          1     0.0526          1     0.0905     0.0452
          Eichel Konig          1          1      0.125          1      0.497      0.348
                Rose 8          1          1          0          0          0          0
              Rose Ass          1          1     0.0435          1      0.332      0.332
            Rose Konig          1          1          0          0          0          0
             Rose Ober          1          1      0.143          1      0.332      0.298
            Schelle 1

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  2,  3,  4,  5,  6, 12, 14, 15, 16, 18, 19, 20, 22, 27, 30, 31, 32, 33, 34, 35])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fae844d23f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0

In [14]:
wandb.finish()

In [25]:
run = wandb.init(
    entity="jassair",
    project="ModelV2",
    name="upload-final-finetuned-yolov11-model-overlapping"
)

model_path = "jassair/yolo11l_1to9_36c_ovlp_250423-191451Finetuned/weights/best.pt"

artifact = wandb.Artifact(
    name="yolov11-finetuned-model-overlapping",
    type="model",
    description="YOLOv11 model trained on synthetic, 1to9, 36c, overlapping cards",
    metadata={"framework": "Ultralytics YOLO", "version": "v11"}
)

artifact.add_file(model_path)
run.log_artifact(artifact)

run.finish()


## Test Evaluation

In [64]:
# # Load best model
model_path = "jassair/yolo11l_1to9_36c_ovlp_250423-191451Finetuned/weights/best.pt"

model = YOLO(model_path)

In [65]:
# Function for random selecting an image from a defined path

def random_test_image(pad_to_square=True):
    test_path = Path("/exchange/dspro2/jassair/r_3to5_36c_ovlp/test/images")
    
    image_files = list(test_path.glob("*.png")) + list(test_path.glob("*.jpg"))
    random_image_path = random.choice(image_files)

    image = Image.open(random_image_path)

    # Convert to RGB if needed
    if image.mode in ("RGBA", "P"):
        image = image.convert("RGB")

    # Resize longest side to 640 while keeping aspect ratio
    width, height = image.size
    scale = 640 / max(width, height)
    new_size = (int(width * scale), int(height * scale))
    image = image.resize(new_size, Image.Resampling.LANCZOS)

    if pad_to_square:
        delta_w = 640 - image.size[0]
        delta_h = 640 - image.size[1]
        padding = (delta_w // 2, delta_h // 2, delta_w - delta_w // 2, delta_h - delta_h // 2)
        image = ImageOps.expand(image, padding, fill=(114, 114, 114))

    return image



In [66]:
# Function to load one image from path with image number
def one_test_image(img_path, num, pad_to_square=True):
    test_path = Path(img_path)
    img_files = list(test_path.glob("*.png")) + list(test_path.glob("*.jpg"))

    img = Image.open(img_files[num])

    # Convert to RGB if needed
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")

    # Resize longest side to 640 while keeping aspect ratio
    width, height = img.size
    scale = 640 / max(width, height)
    new_size = (int(width * scale), int(height * scale))
    img = img.resize(new_size, Image.Resampling.LANCZOS)

    if pad_to_square:
        delta_w = 640 - img.size[0]
        delta_h = 640 - img.size[1]
        padding = (delta_w // 2, delta_h // 2, delta_w - delta_w // 2, delta_h - delta_h // 2)
        img = ImageOps.expand(img, padding, fill=(114, 114, 114))

    return img


In [67]:
# Function to display an image with mathplotlib

def display_img(img, size):
    plt.figure(figsize=(size, size))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [70]:
# Load a random test image and display it
test_image = random_test_image()
display_img(test_image, 6)

In [71]:
# Function for making a detection and classification with the loaded model
# conf = confidence threshold
def classify(model, conf, img):
    results = model(img, conf=conf)
    display_img(results[0].plot(), 6)

In [72]:
results = model(test_image, conf=0.7)


0: 640x640 1 Eichel Ass, 1 Rose Ober, 1 Schilte 7, 1 Schilte Under, 6.5ms
Speed: 2.7ms preprocess, 6.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


In [74]:
results = model(test_image, conf=0.7) # Make one prediction

result = results[0] # 
boxes = result.boxes

for i in range(len(boxes)):
    cls_id = int(boxes.cls[i]) # Card id
    conf = float(boxes.conf[i]) # confidence
    class_name = result.names[cls_id]
    print(cls_id)
    print(conf)
    print(class_name)


0: 640x640 1 Eichel Ass, 1 Rose Ober, 1 Schilte 7, 1 Schilte Under, 14.9ms
Speed: 2.2ms preprocess, 14.9ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)
35
0.9255268573760986
Schilte Under
29
0.8520134091377258
Schilte 7
5
0.834506094455719
Eichel Ass
16
0.7772576212882996
Rose Ober


In [75]:
classify(model, 0.1, test_image)


0: 640x640 2 Eichel Asss, 1 Eichel Under, 1 Rose Ober, 1 Rose Under, 1 Schilte 7, 1 Schilte Under, 14.7ms
Speed: 2.2ms preprocess, 14.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


### Automatic Evaluation on Testset

In [77]:
model.val(data='/exchange/dspro2/jassair/r_3to5_36c_ovlp/data.yaml', split='test')

Ultralytics 8.3.114 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA A16, 14891MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1032.0±182.6 MB/s, size: 44.4 KB)


val: Scanning /exchange/dspro2/jassair/r_3to5_36c_ovlp/test/labels... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<00:00, 690.69it/s]

val: New cache created: /exchange/dspro2/jassair/r_3to5_36c_ovlp/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.24it/s]


                   all          5         19      0.727      0.924      0.995      0.858
              Eichel 6          1          1      0.751          1      0.995      0.995
            Eichel Ass          1          1       0.63          1      0.995      0.895
           Eichel Ober          1          1      0.764          1      0.995      0.796
          Eichel Under          1          1      0.418          1      0.995      0.995
               Rose 10          1          1      0.713          1      0.995      0.895
                Rose 6          1          1      0.424          1      0.995      0.796
                Rose 7          2          2          1      0.706      0.995      0.801
             Rose Ober          1          1          1          0      0.995      0.895
            Rose Under          1          1      0.444          1      0.995      0.697
             Schelle 8          1          1      0.776          1      0.995      0.895
           Schelle As

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 1,  5,  7,  8,  9, 10, 11, 16, 17, 21, 23, 24, 25, 26, 29, 30, 35])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7faf39c50a70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.04